In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
#check the file exists
import MyUtil
stockfolder="/home/wode-user/dataset/Stocks"
files = MyUtil.loadAllSubFolders(stockfolder)
print(len(files))

7195


In [3]:
#load SP 500 stocks

dat= pd.read_csv("data/sp500.csv")
symbols= set(dat['Symbol'].tolist())

In [4]:
def getSymbol(filename):
    return filename.split('/')[-1].split('.')[0].upper()

def parse_returns(fname):
    dat1= pd.read_csv(fname)
    dat1=dat1.tail(501)
    dat1=dat1.set_index(pd.DatetimeIndex(dat1['Date']))
    dat1['logClose']=dat1['Close'].apply(np.log)
    dat1['return'] = dat1['logClose']-dat1['logClose'].shift(1)
    dat1.drop(dat1.index[:1], inplace=True)
    dat2=dat1['return']
    return dat2

In [15]:
data=None
cnt=0
for f in files:
    sym= getSymbol(f)
    if sym.upper() in symbols:
        if cnt==0:
            data=parse_returns(f).to_frame()
        else:
            data=pd.concat([data, parse_returns(f)], axis=1)
            
        data.columns.values[-1]=sym
        cnt+=1
backup=data
data=data.tail(500)
symbols=data.columns.values
data=data.fillna(0)
data=data.T

In [18]:

covv=np.corrcoef(data)
covdf = pd.DataFrame(covv)
covdf=covdf.rename({i:v for i,v in enumerate(symbols)}, axis=1)
covdf["Symbol"]=symbols
covdf=covdf.set_index("Symbol")

In [19]:
covdf.to_csv("data/sp500_cov.csv")